In [44]:
import numpy as np
import pandas as pd

In [45]:
import prepostprocessing.cleaning as cleaning
import prepostprocessing.pre_processing as preproc
from qapf import qapf
from qapf import cipw

___

# CIPWFULL normalized data

## Convert to CIPWFULL format

**I used an arbitrary version of the mineralogical data, not sure if this is the correct one** -->is a good one

In [46]:
mineralogy_original = pd.read_excel("../_CIPW/CIPW/AREA1a/interpolated_data.xlsx", index_col=0)
# Only iterested in sample number and oxides
mineralogy = mineralogy_original[['SiO2', 'TiO2', 'Al2O3', 'Fe2O3','FeO', 'MnO', 'MgO', 'CaO', 'Na2O', 'K2O', 'oth']]
mineralogy.columns

Index(['SiO2', 'TiO2', 'Al2O3', 'Fe2O3', 'FeO', 'MnO', 'MgO', 'CaO', 'Na2O',
       'K2O', 'oth'],
      dtype='object')

In [47]:
mineralogy = mineralogy.dropna()

In [48]:
# Check for closed sum
assert all(np.isclose(mineralogy.sum(axis=1), 100.0))

In [49]:
# Remove 'l.i.'' and 'oth' columns and immediately normalize data
mineralogy = preproc.normalize(mineralogy.drop(["oth"], axis=1))

In [50]:
# Check for closed sum
assert all(np.isclose(mineralogy.sum(axis=1), 100.0))

In [51]:
mineralogy.head()

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
8200,71.349288,0.259314,14.790411,0.873382,2.078522,0.053365,0.748670,1.664401,4.350566,3.832082
8201,71.349544,0.259287,14.790208,0.873504,2.078576,0.053377,0.748570,1.664224,4.350587,3.832124
8202,71.349807,0.259260,14.790010,0.873621,2.078621,0.053388,0.748471,1.664050,4.350606,3.832167
8203,71.350074,0.259234,14.789817,0.873732,2.078658,0.053399,0.748373,1.663877,4.350625,3.832212
8204,71.350345,0.259208,14.789630,0.873837,2.078687,0.053409,0.748277,1.663708,4.350643,3.832257


In [52]:
mineralogy

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
8200,71.349288,0.259314,14.790411,0.873382,2.078522,0.053365,0.748670,1.664401,4.350566,3.832082
8201,71.349544,0.259287,14.790208,0.873504,2.078576,0.053377,0.748570,1.664224,4.350587,3.832124
8202,71.349807,0.259260,14.790010,0.873621,2.078621,0.053388,0.748471,1.664050,4.350606,3.832167
8203,71.350074,0.259234,14.789817,0.873732,2.078658,0.053399,0.748373,1.663877,4.350625,3.832212
8204,71.350345,0.259208,14.789630,0.873837,2.078687,0.053409,0.748277,1.663708,4.350643,3.832257
...,...,...,...,...,...,...,...,...,...,...
240233,71.814031,0.245712,14.720025,0.887698,1.843743,0.047073,0.665678,1.513346,4.347693,3.915002
240234,71.813054,0.245879,14.720645,0.886260,1.844266,0.047012,0.666215,1.514317,4.347528,3.914823
240235,71.812095,0.246045,14.721262,0.884840,1.844770,0.046952,0.666746,1.515277,4.347364,3.914648
240236,71.811156,0.246209,14.721876,0.883443,1.845253,0.046892,0.667269,1.516222,4.347202,3.914477


In [53]:
mineralogy1 = mineralogy.iloc[0:45000]
mineralogy2 = mineralogy.iloc[45000:90000]
mineralogy3 = mineralogy.iloc[90000:140000]

In [54]:
cipw.convert_to_CIPWFULL_format(mineralogy1, "../_CIPW/CIPW/AREA1a/area1a_1.txt", dataset_name="Vistelius (1995)")
cipw.convert_to_CIPWFULL_format(mineralogy2, "../_CIPW/CIPW/AREA1a/area1a_2.txt", dataset_name="Vistelius (1995)")
cipw.convert_to_CIPWFULL_format(mineralogy3, "../_CIPW/CIPW/AREA1a/area1a_3.txt", dataset_name="Vistelius (1995)")

_____

## Extract results from CIPWFULL results file

**I only selected quartz, plagioclase and alkali feldspar here as they are of interest to us for the QAPF classification**  
**Feel free to extract addiotnal minerals if they would be of any interest to you**

**Not sure what to do with albite in the final assignments as by the book it should be accounted for as an alkali feldspar and thus be included in the sum of plagioclase but then the resulting plagioclase contents seem quite low.   
This is something you might look into although the method used here seems to be correct according to te resulting QAPF diagram.**

--> indeed when reading the paper of Streckeisen (and other sources), albite should be accounted for as an alkali feldspar, also saw that the datapoints were generally located more to the right side (plagioclase), maybe an overestimation of plagioclase?

--> I changed the settings below so that albite belongs to the alkali feldspars, Think this figure should be more correct according to the papers, but the cotent of plagioclase is indeed quite low. 

In [56]:
df_final1 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA1a/area1a_1_results_excel.txt")
df_final2 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA1a/area1a_2_results_excel.txt")
df_final3 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA1a/area1a_3_results_excel.txt")

In [57]:
df_final = pd.concat([df_final1, df_final2, df_final3])

In [58]:
df_final

,Q,P,K
SAMPLE,,,
8200,25.432531,41.386319,26.325327
8201,25.433014,41.385580,26.325580
8202,25.433466,41.384885,26.325892
8203,25.434012,41.384117,26.326137
8204,25.434465,41.383425,26.326450
...,...,...,...
240233,26.240892,40.615063,26.812876
240234,26.237848,40.618656,26.811676
240235,26.234905,40.622124,26.810527


## QAPF classification

In [59]:
df_final["QAPF"] = qapf.check_QAPF_region(df_final)

In [60]:
df_final["QAPF"].unique()

array(['monzo granite', 'granodiorite',
       'quartz monzodiorite\nquartz monzogabbro'], dtype=object)

In [61]:
df_final.to_excel("../_CIPW/CIPW/AREA1a/qapf.xlsx", index=True)

-------

In [62]:
df_final.iloc[:, 3].value_counts().to_excel("../_CIPW/CIPW/AREA1a/QAPF_counts.xlsx")

-----

In [63]:
QAPF_control = pd.read_excel("../_CIPW/CIPW/1_normalized.xlsx", index_col=0)
Area = pd.read_excel("../_INTERPOLATION/area1_a.xlsx", index_col=0, usecols = [0, 1, 2, 3, 4, 29])

In [64]:
QAPF_control = pd.concat([Area, QAPF_control], axis = 1)
QAPF_control = QAPF_control.dropna()
QAPF_control

,Y_UTM,X_UTM,ZoneNumber,ZoneLetter,area,Q,P,K,QAPF
55,601309.791636,5.939193e+06,50.0,U,11.0,33.571106,31.095099,29.377416,monzo granite
252,621163.761416,5.932229e+06,50.0,U,11.0,32.642693,31.301171,30.237582,monzo granite
285,606817.419976,5.947659e+06,50.0,U,11.0,32.918510,35.473568,26.199190,monzo granite
334,623410.052093,5.923945e+06,50.0,U,11.0,31.612642,33.856539,30.854933,monzo granite
555,623300.653240,5.927630e+06,50.0,U,11.0,33.379299,27.158899,35.613343,monzo granite
...,...,...,...,...,...,...,...,...,...
4451,656014.701155,5.892117e+06,50.0,U,11.0,6.960453,56.531862,19.012866,quartz monzodiorite\nquartz monzogabbro
4483,673932.649313,5.935732e+06,50.0,U,11.0,7.478934,55.494831,18.714255,quartz monzodiorite\nquartz monzogabbro
4501,656517.983051,5.893837e+06,50.0,U,11.0,3.693652,57.670954,18.292837,monzodiorite monzogabbro
4574,620420.284670,5.932198e+06,50.0,U,11.0,2.757353,53.798825,16.919227,monzodiorite monzogabbro


In [65]:
QAPF_control.to_excel("../_CIPW/CIPW/AREA1a/QAPF_control_points.xlsx", index=True)